<h2>Model Selection 모듈</h2>
<h3>교차 검증</h3>
고정된 학습 데이터와 테스트 데이터로 평가를 하다보면 테스트 데이터에만 최적의 성능을 발휘할 수 있도록 편향되게 모델을 유도하는 경향이 생김, 이러한 경우를 과적합이라고 하는데 이 때 다른 테스트 데이터가 들어올 경우에는 성능이 저하됨 따라서 교차 검증을 이용해 더 다양한 학습과 평가를 수행<br><br>
교차 검증은 이러한 데이터 편중을 막기 위해 학습 데이터 세트를 학습 데이터 세트와 검증 데이터 세트로 분리해서 학습과 평가를 수행하는 것
<h4>K 폴드 교차 검증</h4>
K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법<br>
K개의 예측 평가를 구했으면 이를 평균해서 K 폴드 평가 결과로 반영하면 됨

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.datasets import load_iris
import numpy as np

iris= load_iris()
features= iris.data
label= iris.target
dt_clf= DecisionTreeClassifier(random_state=156)

#5개의 폴드 세트로 분리하는 KFold 객체와 폴드별 정확도 담을 리스트 객체
kfold= KFold(n_splits=5)
cv_accuracy=[]
print('붓꽃 데이터 크기:',features.shape[0])

n_iter=0

#KFold 객체의 split을 호출하면 폴드별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index in kfold.split(features):
    #kfold.split으로 반환된 인덱스를 이용해 학습용, 검증용 데이터 추출
    x_train, x_test= features[train_index], features[test_index]
    y_train, y_test= label[train_index], label[test_index]
    #학습 및 예측
    dt_clf.fit(x_train, y_train)
    pred= dt_clf.predict(x_test)
    n_iter+=1
    #반복시 정확도 측정
    accuracy= np.round(accuracy_score(y_test, pred), 4)
    train_size= x_train.shape[0]
    test_size= x_test.shape[0]
    print('\n{0} #교차 검증 정확도 :{1}, 학습 데이터 크기:{2}, 검증 데이터 크기:{3}'.format(n_iter, accuracy, train_size, test_size))
    print('# {0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
#개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n #평균 정확도:',np.mean(cv_accuracy))

붓꽃 데이터 크기: 150

1 #교차 검증 정확도 :1.0, 학습 데이터 크기:120, 검증 데이터 크기:30
# 1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

2 #교차 검증 정확도 :0.9667, 학습 데이터 크기:120, 검증 데이터 크기:30
# 2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

3 #교차 검증 정확도 :0.8667, 학습 데이터 크기:120, 검증 데이터 크기:30
# 3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

4 #교차 검증 정확도 :0.9333, 학습 데이터 크기:120, 검증 데이터 크기:30
# 4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

5 #교차 검증 정확도 :0.7333, 학습 데이터 크기:120, 검증 데이터 크기:30
# 5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

 #평균 정확도: 0.9


<h4>Stratified K 폴드</h4>
불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식<br>

In [4]:
#K폴드가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배하지 못하는 경우의 문제
import pandas as pd

iris= load_iris()
iris_df= pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']= iris.target
iris_df['label'].value_counts()

kfold= KFold(n_splits=3)
n_iter=0
for train_index, test_index in kfold.split(iris_df):
    n_iter+=1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차검증 :{0}'.format(n_iter))
    print('학습 레이블 \n', label_train.value_counts())
    print(' 검증 레이블 \n', label_test.value_counts())

## 교차검증 :1
학습 레이블 
 1    50
2    50
Name: label, dtype: int64
 검증 레이블 
 0    50
Name: label, dtype: int64
## 교차검증 :2
학습 레이블 
 0    50
2    50
Name: label, dtype: int64
 검증 레이블 
 1    50
Name: label, dtype: int64
## 교차검증 :3
학습 레이블 
 0    50
1    50
Name: label, dtype: int64
 검증 레이블 
 2    50
Name: label, dtype: int64


In [5]:
#StratifiedKfold kfold로 분할된 레이블 데이터 세트가 전체 레이블 값의 분포도를 반영하지 못하는 문제 해결
#split메서드에 인자로 피처 데이터 세트 분아니라 레이블 데이터 세트도 필요
from sklearn.model_selection import StratifiedKFold

skf= StratifiedKFold(n_splits=3)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter+=1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증:{0}'.format(n_iter))
    print('학습용 데이터:\n',label_train.value_counts())
    print('검증용 데이터:\n', label_test.value_counts())

#학습 레이블과 검증 레이블 데이터 값의 부노도가 동일하게 할당 되었음

## 교차 검증:1
학습용 데이터:
 2    34
0    33
1    33
Name: label, dtype: int64
검증용 데이터:
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차 검증:2
학습용 데이터:
 1    34
0    33
2    33
Name: label, dtype: int64
검증용 데이터:
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차 검증:3
학습용 데이터:
 0    34
1    33
2    33
Name: label, dtype: int64
검증용 데이터:
 1    17
2    17
0    16
Name: label, dtype: int64


In [6]:
#StratifiedKFold로 붓꽃 데이터 교차 검증
dt_clf= DecisionTreeClassifier(random_state=156)
skfold= StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

#StratifiedKFold split 호출시 반드시 레이블 데이터 세트도 추가 입력 필요
for train_index, test_index in skfold.split(features, label):
    #split으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    x_train, x_test= features[train_index], features[test_index]
    y_train, y_test= label[train_index], label[test_index]
    #학습 및 예측
    dt_clf.fit(x_train, y_train)
    pred= dt_clf.predict(x_test)
    
    #반복 시마다 정확도 측정
    n_iter+=1
    accuracy= np.round(accuracy_score(y_test, pred), 4)
    train_size= x_train.shape[0]
    test_size= x_test.shape[0]
    print('\n{0} 교차 검증 정확도:{1}, 학습 데이터 크기:{2}, 검증 데이터 크기:{3}'.format(n_iter,accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    

#교차 검증별 정확도 및 평균 정확도 계산
print('\n ## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy))


1 교차 검증 정확도:0.98, 학습 데이터 크기:100, 검증 데이터 크기:50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

2 교차 검증 정확도:0.94, 학습 데이터 크기:100, 검증 데이터 크기:50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

3 교차 검증 정확도:0.98, 학습 데이터 크기:100, 검증 데이터 크기:50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

 ## 교차 검증별 정확도: [0.98 0.94 0.98]
## 평균 검증 정확도: 0.9666666666666667


<h4>교차 검증을 보다 간편하게 - cross_val_score()</h4>
폴드 세트를 설정하고 for 루프에서 반복으로 학습 및 테스트 데이터의 인덱스를 추출한뒤 반복적으로 학습과 예측을 수행하고 예측 성능을 반환하는 과정을 한꺼번에 수행<br>

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data= load_iris()
dt_clf= DecisionTreeClassifier(random_state=156)

data= iris_data.data
label= iris_data.target

#성능 지표는 정확도, 교차 검증 세트는 3개
scores= cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도', np.round(scores, 4))
print('평균 검증 정확도', np.round(np.mean(scores),4))

#cross_val_score는 API내부에서 학습, 예측, 평가시켜주므로 간단하게 교차 검증만 수행하면 됨
#cross_validate는 여러개의 평가 지표를 반환할 수 있음

교차 검증별 정확도 [0.98 0.94 0.98]
평균 검증 정확도 0.9667


<h3>GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에</h3>
사이킷런은 GridSearchCV API를 이용해 Classifier나 Regressor와 같은 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있는 방안 제공

In [8]:
grid_parameters={'max_depth':[1, 2, 3],
                'min_samples_split':[2, 3]
                }


GridSearchCV 클래스의 주요 파라미터<br>
estimator: classifier, regressor, pipeline 주로 사용<br>
param_grid: key+리스트 값을 가지는 딕셔너리가 주어짐<br>
scoring: 예측 성능을 측정할 평가 방법을 지정<br>
cv: 교차 검증을 위해 분할되는 학습/데이터 세트의 개수 지정<br>
refit: default=True, 가장 최적의 하이퍼 파라미터를 찾은 후 estimator 객체를 해당 하이퍼 파라미터로 재학습

In [11]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

#데이터 로딩, 학습 데이터와 테스트 데이터 분리
iris_data= load_iris()
x_train, x_test, y_train, y_test= train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=121)

dtree= DecisionTreeClassifier()

###파라미터는 딕셔너리 형태로 설정
parameters={'max_depth':[1, 2, 3],
           'min_samples_split':[2, 3]
           }

import pandas as pd

#param_grid의 하이퍼 파라미터를 3개의 train, test set fold 로 나누어 테스트 수행 설정
grid_dtree= GridSearchCV(dtree, param_grid=parameters, cv=3)

#붓꽃 학습 데이터로 param_grid 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(x_train, y_train)

#GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df= pd.DataFrame(grid_dtree.cv_results_) #cv_results_는 딕셔너리 형태로 key값과 리스트 형태의 value를 가짐
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [13]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

#refit=True이면 최적 성능을 나타내느 하이퍼 파라미터로 Estimator를 학습해 best_estimator_로 저장
#GridSearchCV의 refit으로 이미 학습된 estimator 반환
estimator= grid_dtree.best_estimator_

#GridSearchCV의 best_estimator_는 이미 최적 학습이 되어있음
pred= estimator.predict(x_test)
print('테스트 데이터 세트 정확도:{0:.4f}'.format(accuracy_score(y_test, pred)))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750
테스트 데이터 세트 정확도:0.9667
